In [1]:
import json
from pprint import pp

In [2]:
entries = {};
with open("../output.jsonl") as f:
  for line in f:
    pair = json.loads(line)
    entries[pair[0]] = pair[1]


In [3]:
reflexive_entries = []
for word, entry_list in entries.items():
    if word.endswith('rse'):
        for entry in entry_list:
            if 'form_of' not in entry and entry not in reflexive_entries and entry['pos'] == 'verb' and len(entry['word'].split()) == 1:
                reflexive_entries.append(entry)

reflexive_lemmas = {entry['word'] for entry in reflexive_entries}
print(len(reflexive_entries))
# reflexive_entries
# pp([{"word": entry['word'], "definitions": [entry['definitions']]} for entry in reflexive_entries if not ])
# len([{"word": entry['word'], "definitions": [entry['definitions']]} for entry in reflexive_entries if not all("reflexive" in definition.get("label", "") for definition in entry['definitions'])])
# pp([{"word": entry['word'], "definitions": [entry['definitions']]} for entry in reflexive_entries if not all("reflexive" in definition.get("label", "") for definition in entry['definitions'])])

247


In [4]:
reflexive_definitions = [
  {
    "word": entry['word'],
    "definitions": list((", ".join(list(definition.values()))) for definition in entry['definitions'])
  }
  for entry in reflexive_entries
]

In [5]:
for defin in (reflexive_definitions):
  pp(defin)


{'word': 'ufanarse',
 'definitions': ['transitive with de, to take pride in',
                 'transitive with de, to boast']}
{'word': 'quejarse',
 'definitions': ['to complain, to whine, to gripe, to grouse, to fuss, to '
                 'bitch, to beef',
                 'to grumble, to moan, to groan',
                 'to quarrel']}
{'word': 'querellarse', 'definitions': ['to sue; to bring suit']}
{'word': 'abstenerse', 'definitions': ['reflexive, to abstain, to forbear']}
{'word': 'acalenturarse', 'definitions': ['reflexive, to have a fever']}
{'word': 'carcajearse',
 'definitions': ['reflexive, to laugh heartily or heavily; to cackle; to bust '
                 'up or be in stitches']}
{'word': 'adueñarse', 'definitions': ['to appropriate, to take ownership']}
{'word': 'autodestruirse', 'definitions': ['reflexive, to self-destruct']}
{'word': 'acodarse', 'definitions': ['to lean']}
{'word': 'persignarse',
 'definitions': ['reflexive, Christianity, to cross oneself,, make the s

In [6]:
has_clean_non_combined_form_reflexive_entry = []
has_problematic_non_combined_form_reflexive_entry = []
does_not_have_non_combined_lemma_entry = []
for reflexive_entry in reflexive_entries:
  non_combined_lemma = reflexive_entry.get("word", "")[:-2]
  if entries.get(non_combined_lemma):
    non_combined_entry = entries.get(non_combined_lemma)
    triplet = (
      len(non_combined_entry) == 1,
      len(non_combined_entry[0]['definitions']) == 1,
      "only used in" in non_combined_entry[0]['definitions'][0]['definition']
    )
    if all(triplet):
      # print("in first condition?")
      has_clean_non_combined_form_reflexive_entry.append((reflexive_entry, triplet))
    else:
      # print("in else?", non_combined_entry, non_combined_entry[0]['definitions'][0])
      has_problematic_non_combined_form_reflexive_entry.append((reflexive_entry, non_combined_entry, triplet))
  else:
    does_not_have_non_combined_lemma_entry.append(reflexive_entry)


# print(len([entry for entry in [(entry, entries.get(entry['word'][:-2]), len(entries.get(entry['word'][:-2])) == 1, len(entries.get(entry['word'][:-2])[0]['definitions']) == 1, "only used in" in entries.get(entry['word'][:-2])[0]['definitions'][0]['definition']) for entry in reflexive_entries if entries.get(entry['word'][:-2])] 
#  if any([not x for x in entry[2:]])]))
# pp(([entry for entry in [(entry, entries.get(entry['word'][:-2]), len(entries.get(entry['word'][:-2])) == 1, len(entries.get(entry['word'][:-2])[0]['definitions']) == 1, "only used in" in entries.get(entry['word'][:-2])[0]['definitions'][0]['definition']) for entry in reflexive_entries if entries.get(entry['word'][:-2])] 
#  if any([not x for x in entry[2:]])])[0])

# print(len([entry for entry in [(entry, entries.get(entry['word'][:-2]), len(entries.get(entry['word'][:-2])) == 1, len(entries.get(entry['word'][:-2])[0]['definitions']) == 1, "only used in" in entries.get(entry['word'][:-2])[0]['definitions'][0]['definition']) for entry in reflexive_entries if entries.get(entry['word'][:-2])] 
#  if not any([not x for x in entry[2:]])]))

print("clean:", len(has_clean_non_combined_form_reflexive_entry))
print("problematic:", len(has_problematic_non_combined_form_reflexive_entry))
pp([entry, has_problematic_non_combined_form_reflexive_entry])

clean: 206
problematic: 1
[{'word': 'baladronearse',
  'pos': 'verb',
  'from_forms': True,
  'form_of': 'baladronear',
  'definitions': []},
 [({'word': 'incautarse',
    'pos': 'verb',
    'forms': ['me incauté',
              'incautarme',
              'incautarte',
              'te incautarías',
              'incautaros',
              'incáutese',
              'os incautabais',
              'se incautará',
              'te incautaste',
              'se incautare',
              'se incauten',
              'te incautares',
              'se incautarían',
              'os incautareis',
              'incáutate',
              'os incautarais',
              'se incautasen',
              'incautándonos',
              'te incautes',
              'nos incautáremos',
              'se incautaba',
              'me incautaría',
              'incautándote',
              'os incautéis',
              'incautarse',
              'te incautarás',
              'se incautaran',


so, for 206 (of 247 total reflexive), the non-combined form of the lemma just points right back at the reflexive lemma, 

(and for 1 more it's slightly fussier but we could still trivially wipe out the entry for the non-combined lemma (maybe hard to identify algorithmically))

In [7]:
len(does_not_have_non_combined_lemma_entry)
does_not_have_non_combined_lemma_entry

[{'word': 'petatearse',
  'pos': 'verb',
  'forms': ['me petateé', 'petateado', 'me petateo'],
  'definitions': [{'label': 'colloquial, Chile, El Savador, Mexico, Nicaragua, Panama, takes a reflexive pronoun',
    'definition': 'to die'}]},
 {'word': 'agolparse',
  'pos': 'verb',
  'forms': ['agolpándote',
   'se agolparía',
   'os agolparais',
   'os agolparéis',
   'se agolparán',
   'agolpándome',
   'me agolparía',
   'os agolpaseis',
   'se agolpan',
   'se agolpara',
   'se agolpe',
   'agolpaos',
   'me agolpara',
   'agolparme',
   'agolpada',
   'os agolpáis',
   'os agolpareis',
   'te agolparías',
   'agolparnos',
   'agolparos',
   'nos agolparemos',
   'te agolparas',
   'se agolpa',
   'agolpándose',
   'me agolpare',
   'te agolpabas',
   'agolparte',
   'nos agolpábamos',
   'se agolpó',
   'se agolpare',
   'agólpate',
   'os agolparíais',
   'os agolpasteis',
   'se agolpará',
   'os agolpéis',
   'me agolpaba',
   'me agolpase',
   'nos agolpáramos',
   'agolparse',


In [8]:
def definition_does_have_reflexive_label(definition):
  return "reflexive" in definition.get("label", "")
  print(all(["reflexive" in definition.get("label", "") for definition in entry['definitions']]))

def definition_does_have_reflexive_in_definition(definition):
  return "reflexive" in definition.get("definition", "")

def entry_does_have_reflexive_label_on_some_definition(entry):
  return any(definition_does_have_reflexive_label(definition) or definition_does_have_reflexive_in_definition(definition) for definition in entry['definitions'])

In [9]:
does_not_have_reflexive_label = [entry for entry in reflexive_entries if not entry_does_have_reflexive_label_on_some_definition(entry)]
print(len(does_not_have_reflexive_label))
has_reflexive_on_definition = [entry for entry in reflexive_entries if any(definition_does_have_reflexive_in_definition(definition) for definition in entry['definitions'])]
print(len(has_reflexive_on_definition))
pp(has_reflexive_on_definition)

82
1
[{'word': 'enchicharse',
  'pos': 'verb',
  'forms': ['enchichado', 'me enchicho', 'me enchiché'],
  'definitions': [{'label': 'Latin America',
                   'definition': '(reflexive of enchichar) to get drunk on '
                                 'chicha'},
                  {'label': 'Andes, Colombia, Central America',
                   'definition': 'to become angry'}]}]


In [10]:
does_not_have_reflexive_label

[{'word': 'ufanarse',
  'pos': 'verb',
  'forms': ['se ufanase',
   'te ufanaste',
   'se ufanare',
   'ufanadas',
   'ufanarnos',
   'te ufanes',
   'os ufanabais',
   'me ufane',
   'os ufanáis',
   'me ufanaba',
   'ufanada',
   'se ufanará',
   'se ufanaba',
   'te ufanarás',
   'ufanándome',
   'se ufanaran',
   'ufanaos',
   'ufanados',
   'nos ufanáremos',
   'ufánese',
   'me ufanaré',
   'ufanándose',
   'ufanándonos',
   'se ufana',
   'te ufanarías',
   'ufánate',
   'te ufanaras',
   'nos ufanábamos',
   'os ufanaréis',
   'se ufanaría',
   'os ufanaríais',
   'te ufanabas',
   'se ufanasen',
   'nos ufanemos',
   'ufanándoos',
   'se ufanan',
   'nos ufanaríamos',
   'ufanaros',
   'me ufané',
   'se ufanaren',
   'ufanate',
   'ufanémonos',
   'os ufanarais',
   'ufanarse',
   'te ufanás',
   'nos ufanamos',
   'os ufanasteis',
   'se ufane',
   'os ufanéis',
   'ufanarme',
   'se ufanó',
   'ufanarte',
   'me ufanaría',
   'ufánense',
   'se ufanarán',
   'se ufanen',
  

In [11]:
reflexive_lemmas_with_forms_pointing_to_them = set()
form_of_points_to_reflexive = []
for word_entry in entries.values():
  for entry in word_entry:
    if entry.get("form_of") in reflexive_lemmas:
      form_of_points_to_reflexive.append(entry)
      reflexive_lemmas_with_forms_pointing_to_them.add(entry.get("form_of"))

In [12]:
print(len(form_of_points_to_reflexive))
print(len(reflexive_lemmas_with_forms_pointing_to_them))
print(reflexive_lemmas_with_forms_pointing_to_them == reflexive_lemmas)


27325
247
True


In [13]:
pp(form_of_points_to_reflexive[:10])

[{'word': 'muse',
  'pos': 'verb',
  'form_of': 'musirse',
  'definitions': [{'definition': 'only used in se muse, third-person singular '
                                 'present indicative of musirse'},
                  {'definition': 'only used in te ... muse, syntactic variant '
                                 'of músete, second-person singular imperative '
                                 'of musirse'}]},
 {'word': 'jambe',
  'pos': 'verb',
  'form_of': 'jambarse',
  'definitions': [{'definition': 'only used in me jambe, first-person singular '
                                 'present subjunctive of jambarse'},
                  {'definition': 'only used in se jambe, third-person singular '
                                 'present subjunctive of jambarse'},
                  {'definition': 'only used in se ... jambe, syntactic variant '
                                 'of jámbese, third-person singular imperative '
                                 'of jambarse'}]},
 {'word':

In [14]:
complete_reflexive_entries = [word_entry for word_entry in entries.values() if any(
  entry.get("word") in reflexive_lemmas for entry in word_entry
)]
print(len(complete_reflexive_entries))
print(all([len(entry) == 1 for entry in complete_reflexive_entries]))

247
True


so *currently* these all just have a single one per entry
but that's not guaranteed to stay that way? 

(and actually this is from kaikki as of ~2024-05)

hm


In [51]:
def multi_token_forms_contains_new_forms(entry):
  forms = entry.get("forms", [])
  if not forms:
    return False
  single_token_forms, multi_token_forms = [], []
  for form in forms:
    split = form.split()
    if len(split) > 1:
      multi_token_forms.append(form)
    else:
      single_token_forms.append(form)
  second_part_of_multi_tokens_set = {token.split()[1] for token in multi_token_forms}
  single_token_forms_set = set(single_token_forms)
  second_part_of_multi_tokens_not_in_single_tokens = {x for x in second_part_of_multi_tokens_set if x not in single_token_forms_set}
  single_token_forms_not_in_second_part_of_multi_tokens_set = {x for x in single_token_forms_set if x not in second_part_of_multi_tokens_set}
  pp(locals())


multi_token_forms_contains_new_forms(entries['autoconstruirse'][0]);


{'entry': {'word': 'autoconstruirse',
           'pos': 'verb',
           'forms': ['me autoconstruyese',
                     'nos autoconstruyamos',
                     'se autoconstruya',
                     'te autoconstruyeres',
                     'me autoconstruya',
                     'autoconstruirnos',
                     'se autoconstruían',
                     'nos autoconstruiríamos',
                     'autoconstruidos',
                     'autoconstruíos',
                     'te autoconstruías',
                     'se autoconstruye',
                     'se autoconstruyera',
                     'autoconstrúyase',
                     'autoconstruirte',
                     'te autoconstruyes',
                     'autoconstruite',
                     'se autoconstruirán',
                     'me autoconstruiré',
                     'autoconstruirse',
                     'os autoconstruyerais',
                     'autoconstruyéndonos',
            

ok so trivially they do actually contain meaningful forms in the reflexive forms

...I feel like I noticed this way back in may and convinced myself they didn't matter? idk

In [79]:
all_multi_token_verb_forms = [
  {
    "word": entry['word'],
    "multi_token_forms": [form for form in entry['forms'] if len(form.split()) > 1],
    "forms": entry['forms'],
    "forms_equals_multi_token_forms": [form for form in entry['forms'] if len(form.split()) > 1] == entry['forms']
  } for word_entry in entries.values()
  for entry in word_entry
  if entry['pos'] == 'verb' and entry.get("forms") and any(
    len(form.split()) > 1 for form in entry.get("forms")
  )
]
# all_first_entries = {form.split()[0] for entry in all_multi_token_verb_forms for form in entry['multi_token_forms']}
multi_token_verb_forms_with_more_than_two_tokens_in_some_form = [
  entry for entry in all_multi_token_verb_forms
  if any(len(form.split()) > 2 for form in entry['multi_token_forms'])
]
multi_token_verb_forms_with_more_than_two_tokens_in_some_form[::10]

[{'word': 'dar a luz',
  'multi_token_forms': ['dado a luz', 'di a luz', 'doy a luz'],
  'forms': ['dado a luz', 'di a luz', 'doy a luz'],
  'forms_equals_multi_token_forms': True},
 {'word': 'haigan',
  'multi_token_forms': ['used as a joke'],
  'forms': ['used as a joke'],
  'forms_equals_multi_token_forms': True},
 {'word': 'tomar en cuenta',
  'multi_token_forms': ['tomado en cuenta',
   'tomé en cuenta',
   'tomo en cuenta'],
  'forms': ['tomado en cuenta', 'tomé en cuenta', 'tomo en cuenta'],
  'forms_equals_multi_token_forms': True},
 {'word': 'echar a perder',
  'multi_token_forms': ['echo a perder', 'echado a perder', 'eché a perder'],
  'forms': ['echo a perder', 'echado a perder', 'eché a perder'],
  'forms_equals_multi_token_forms': True},
 {'word': 'meter la cuchara',
  'multi_token_forms': ['meto la cuchara',
   'metido la cuchara',
   'metí la cuchara'],
  'forms': ['meto la cuchara', 'metido la cuchara', 'metí la cuchara'],
  'forms_equals_multi_token_forms': True},
 {'

In [71]:
[entry for entry in all_multi_token_verb_forms if "pasándolas" in [form.split()[0] for form in entry['multi_token_forms']]]

[{'word': 'pasar las de Caín',
  'multi_token_forms': ['pasadme las de Caín',
   'pasaríais las de Caín',
   'pasáremos las de Caín',
   'pasarás las de Caín',
   'páseme las de Caín',
   'pasemos las de Caín',
   'pasémosla las de Caín',
   'pasadle las de Caín',
   'pasá las de Caín',
   'pásalos las de Caín',
   'pasándose las de Caín',
   'pasándonos las de Caín',
   'pasaseis las de Caín',
   'pásalo las de Caín',
   'pasabais las de Caín',
   'páselo las de Caín',
   'pasarían las de Caín',
   'pasémoslos las de Caín',
   'pasábamos las de Caín',
   'pasadla las de Caín',
   'pásense las de Caín',
   'pasés las de Caín',
   'pasada las de Caín',
   'pasarse las de Caín',
   'pasanos las de Caín',
   'pasalas las de Caín',
   'pasaremos las de Caín',
   'pasareis las de Caín',
   'pasadnos las de Caín',
   'pasarlos las de Caín',
   'pasaban las de Caín',
   'pasaste las de Caín',
   'pásales las de Caín',
   'pasémoslas las de Caín',
   'pasásemos las de Caín',
   'pasares las de

In [72]:
entries['pasar las de Caín']

[{'word': 'pasar las de Caín',
  'pos': 'verb',
  'forms': ['pasadme las de Caín',
   'pasaríais las de Caín',
   'pasáremos las de Caín',
   'pasarás las de Caín',
   'páseme las de Caín',
   'pasemos las de Caín',
   'pasémosla las de Caín',
   'pasadle las de Caín',
   'pasá las de Caín',
   'pásalos las de Caín',
   'pasándose las de Caín',
   'pasándonos las de Caín',
   'pasaseis las de Caín',
   'pásalo las de Caín',
   'pasabais las de Caín',
   'páselo las de Caín',
   'pasarían las de Caín',
   'pasémoslos las de Caín',
   'pasábamos las de Caín',
   'pasadla las de Caín',
   'pásense las de Caín',
   'pasés las de Caín',
   'pasada las de Caín',
   'pasarse las de Caín',
   'pasanos las de Caín',
   'pasalas las de Caín',
   'pasaremos las de Caín',
   'pasareis las de Caín',
   'pasadnos las de Caín',
   'pasarlos las de Caín',
   'pasaban las de Caín',
   'pasaste las de Caín',
   'pásales las de Caín',
   'pasémoslas las de Caín',
   'pasásemos las de Caín',
   'pasares l

In [62]:
print(len(all_first_entries))
print(all_first_entries)

4016
{'chingado', 'levantaren', 'pasándolas', 'átale', 'comulgo', 'pasaríamos', 'sacó', 'ofendo', 'emítame', 'renovo', 'cogedme', 'tomémonos', 'téngales', 'delo', 'formé', 'leemos', 'teniéndole', 'bailado', 'transmitiera', 'acertaríamos', 'cogiereis', 'atenido', 'apretabais', 'estado', 'mandé', 'lo', 'bailadlas', 'bailarla', 'leeréis', 'ateniéndose', 'chingaran', 'estaba', 'costado', 'chingarla', 'volares', 'apretá', 'chinguémoslas', 'emitirlas', 'trabado', 'importado', 'tengáis', 'refrescado', 'levantaste', 'sácate', 'emitiéndome', 'apretaseis', 'echarían', 'echanos', 'hacerte', 'cogeréis', 'atándonos', 'apretate', 'cójanse', 'surtirle', 'bailábamos', 'chingarías', 'báilenme', 'irse', 'tírale', 'duermo', 'sacándolas', 'chingarnos', 'tíreme', 'disteis', 'sedlos', 'pasaras', 'levantados', 'pasarlas', 'tirándoles', 'chingándolas', 'tuviéremos', 'ganados', 'tíreles', 'estándome', 'estarlas', 'cogíais', 'hurgado', 'pasamos', 'emítanlos', 'tomá', 'disparado', 'chingándote', 'pasalas', 'tran

ok wellllll oy vey
it's not that many maybe tho

In [65]:
[entry for entry in all_multi_token_verb_forms if "pasándolas" in entry.get("multi_token_forms")]

[]

In [ ]:
complete_reflexive_entries = [word_entry for word_entry in entries.values() if any(
  entry.get("word") in reflexive_lemmas for entry in word_entry
)]
print(len(complete_reflexive_entries))
print(all([len(entry) == 1 for entry in complete_reflexive_entries]))

247
True


In [30]:
multi_token_form_of_not_from_reflexives = [
  entry for word_entry in entries.values()
  for entry in word_entry
  if len(entry['word'].split()) > 1 and
  entry.get("form_of") and
  entry.get("form_of") not in reflexive_lemmas
]
multi_token_verb_form_of_not_from_reflexives = [
  entry for entry in multi_token_form_of_not_from_reflexives
  if entry['pos'] == 'verb'
]

In [39]:
print(len({entry['word'] for entry in multi_token_verb_form_of_not_from_reflexives}))
print(len({entry['form_of'] for entry in multi_token_verb_form_of_not_from_reflexives}))
print(len({entry['word'] for entry in multi_token_verb_form_of_not_from_reflexives if not entry.get("from_forms")}))
print(len({entry['form_of'] for entry in multi_token_verb_form_of_not_from_reflexives if not entry.get("from_forms")}))


14252
1833
40
36


ok so in fact there are quite a few that aren't from reflexives in the current config, including a handful that aren't even `from_forms`'s

In [40]:
[entry for entry in multi_token_verb_form_of_not_from_reflexives if not entry.get("from_forms")]

[{'word': 'armar un san Quintín',
  'pos': 'verb',
  'forms': ['armado un san Quintín',
   'armé un san Quintín',
   'armo un san Quintín'],
  'form_of': 'armar la de san Quintín',
  'definitions': [{'definition': 'Rare form of armar la de san Quintín.'}]},
 {'word': 'llegar a final de mes',
  'pos': 'verb',
  'forms': ['llegué a final de mes',
   'llego a final de mes',
   'llegado a final de mes'],
  'form_of': 'llegar a fin de mes',
  'from_alt_of': True,
  'definitions': [{'label': 'idiomatic',
    'definition': 'Alternative form of llegar a fin de mes'}]},
 {'word': 'darse de baja',
  'pos': 'verb',
  'forms': ['me di de baja', 'me doy de baja', 'dado de baja'],
  'form_of': 'dar de baja',
  'definitions': [{'definition': 'reflexive of dar de baja: to unsubscribe, deregister'}]},
 {'word': 'darse de alta',
  'pos': 'verb',
  'forms': ['me doy de alta', 'me di de alta', 'dado de alta'],
  'form_of': 'dar de alta',
  'definitions': [{'definition': 'reflexive of dar de alta; to disch